In [1]:
from kdp import *

Let's build a graph from a region that's super simple (hom, both matching) first.

We'll have the H (haplotypes (or reads) graph) and G (graph variants to be phased/genotyped).

Starting at the snk of H - we'll have two options, use it, or don't use it.

For the use it, we check every neighboring node for G from snk and record the sizediff and cossim.
    if the sizediff and cossim is above thresholds,
        We apply Hn to Gn, update our running Hsk and Gsk (size and kmer features of the paths being joined).

if we can't use it, well.. because we have in the graph snk->everyNode, we know we've checked Use it and don't use it.
So, not using it is just saying, 'well go to the next node'.

Okay, we we have two things.
    Hi and Gi
    And we try to apply each Hi+1 to Gi+1,
    If none of them are good enough... we need to try and both Hi+1 and Hi+2 as well as just moving to Hi+2.
    That's the problem here.
    
    
```
Initialize dp[] with appropriate initial values

for each node a in graph A's full path:
    for each node b in graph B:
        if b's parent has been visited in graph A's path:
            dp[b] = max(dp[b], dp[b's parent] + weight of (b's parent to b))

Find the node b_last with the maximum value in dp[]
Backtrack from b_last to find the actual path in graph B
```

 

In [ ]:
import numpy as np
m_dp = np.zeros(len(H.nodes), len(G.nodes))

for h_i, h_n in enumerate(H.nodes):
    for g_i, g_n in enumerate(G.nodex):
        #m_dp[h_i][g_i] = max(dp[b], dp[b's parent] + weight of (b's parent to b))

        